In [4]:
#IMPORTING THE NECCESSARY LIBRARIES 
import os
import sys
import math
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import spacy
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [5]:
#CHECKING FOR GPU AVAILIBILITY
!nvidia-smi


Wed Nov 24 15:54:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#REDUCING THE SIZE O0D DATASET
df = pd.read_csv('/content/drive/MyDrive/nlp-project /qg_valid.csv', nrows=10000)
df.to_csv('/content/drive/MyDrive/nlp-project /newvalid.csv')

In [ ]:
#INSTALLING NECCSSARY LIBRARIES 
pip install -r /content/question_generator/requirements.txt

     |████████████████████████████████| 883 kB 8.7 MB/s 
     |████████████████████████████████| 1.1 MB 49.3 MB/s 
     |████████████████████████████████| 9.9 MB 15.0 MB/s 
     |████████████████████████████████| 2.9 MB 68.0 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 1.5 MB 93.2 MB/s 
     |████████████████████████████████| 2.1 MB 83.5 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893250 sha256=222375fb2f197e1db716fa99244a0301a18c81c8bb36df9d37aa7d1c2b9ea176
  Stored in directory: /root/.cache/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built sacremoses
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled 

In [ ]:
!pip install transformers

     |████████████████████████████████| 3.1 MB 5.2 MB/s 
     |████████████████████████████████| 59 kB 8.2 MB/s 
     |████████████████████████████████| 3.3 MB 53.5 MB/s 
     |████████████████████████████████| 895 kB 66.5 MB/s 
     |████████████████████████████████| 596 kB 69.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [6]:
#IMNPORTING T5TOKENIZER, T5CONDITIONAL GENERATION AND T5CONFIG FILE
from transformers import T5Tokenizer, T5ForConditionalGeneration, T5Config

In [7]:
#DEVICE CHECKING
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [2]:
#CLONING THE GITHUB GIT TO USE SOME PREWRITTEN CODE
!git clone https://github.com/AMontgomerie/question_generator.git

Cloning into 'question_generator'...
remote: Enumerating objects: 202, done.
remote: Counting objects: 100% (90/90), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 202 (delta 47), reused 22 (delta 9), pack-reused 112
Receiving objects: 100% (202/202), 102.36 KiB | 888.00 KiB/s, done.
Resolving deltas: 100% (102/102), done.


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.2 MB 5.1 MB/s 


In [3]:
!pip install -r /content/question_generator/requirements.txt

     |████████████████████████████████| 883 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 75.9 MB/s 
     |████████████████████████████████| 9.9 MB 29.3 MB/s 
     |████████████████████████████████| 2.9 MB 65.7 MB/s 
     |████████████████████████████████| 776.8 MB 17 kB/s 
     |████████████████████████████████| 1.5 MB 69.4 MB/s 
     |████████████████████████████████| 2.1 MB 20.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-py3-none-any.whl size=893250 sha256=7d9cee46b3c5d5839d279c5059aa793013774a61563942e8b20b51e9160105f1
  Stored in directory: /root/.cache/pip/wheels/69/09/d1/bf058f7d6fa0ecba2ce7c66be3b8d012beb4bf61a6e0c101c0
Successfully built sacremoses
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled 

In [8]:
#PREPARING DATASET FOR TRAINGING AND VALIDATION
PRETRAINED_MODEL = 't5-base'
DIR = "question_generator/"
BATCH_SIZE = 2
SEQ_LENGTH = 512

tokenizer = T5Tokenizer.from_pretrained(PRETRAINED_MODEL)
tokenizer.add_special_tokens(
    {'additional_special_tokens': ['<answer>', '<context>']}
)

class QGDataset(Dataset):
    def __init__(self, csv):
        self.df = pd.read_csv(csv, engine='python')

    def __len__(self):
         return len(self.df)

    def __getitem__(self, idx):   
        if torch.is_tensor(idx):
            idx = idx.tolist()
        row = self.df.iloc[idx, 1:]       

        encoded_text = tokenizer(
            row['text'], 
            pad_to_max_length=True, 
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors="pt"
        )
        encoded_text['input_ids'] = torch.squeeze(encoded_text['input_ids'])
        encoded_text['attention_mask'] = torch.squeeze(encoded_text['attention_mask'])

        encoded_question = tokenizer(
            row['question'],
            pad_to_max_length=True,
            max_length=SEQ_LENGTH,
            truncation=True,
            return_tensors='pt'
        )
        encoded_question['input_ids'] = torch.squeeze(encoded_question['input_ids'])

        return (encoded_text.to(device), encoded_question.to(device))

train_set = QGDataset(os.path.join(DIR, '/content/drive/MyDrive/nlp-project /newtrain.csv'))
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
valid_set = QGDataset(os.path.join(DIR, '/content/drive/MyDrive/nlp-project /newvalid.csv')) 
valid_loader = DataLoader(valid_set, batch_size=BATCH_SIZE, shuffle=False)

Downloading:   0%|          | 0.00/792k [00:00<?, ?B/s]

In [9]:
#DEFINING THE PARAMETERS OF MODEL
LR = 0.001
EPOCHS = 20
LOG_INTERVAL = 5000

config = T5Config(decoder_start_token_id=tokenizer.pad_token_id)
#model = torch.load('/content/drive/MyDrive/nlp-project /qg_pretrained_t5_model_trained.pth')
model = T5ForConditionalGeneration(config).from_pretrained(PRETRAINED_MODEL)
model.resize_token_embeddings(len(tokenizer)) # to account for new special tokens
model = model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=LR)

Downloading:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/892M [00:00<?, ?B/s]

Some weights of the model checkpoint at t5-base were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [10]:
#THIS CELL IS FOR PRELOADING SAVE MODEL TO START TRAINING WHERE YOU LEFT LAST TIME IF WE ARE NOT ABLE TO COMPLETE TRAINING INJ ONE SINGLE RUN
checkpoint = torch.load('/content/drive/MyDrive/nlp-project /qg_pretrained_t5_model_trained.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['best_loss']
#model.eval()

In [ ]:
model.load_state_dict(torch.load('/content/drive/MyDrive/nlp-project /qg_pretrained_t5_model_trained.pth'))

RuntimeError: ignored

In [11]:
#TO SEE MODEL
model

T5ForConditionalGeneration(
  (shared): Embedding(32102, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32102, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [ ]:
#TRAINING AND EVALUATING LOOP
SAVED_MODEL_PATH = "/content/drive/MyDrive/nlp-project /qg_pretrained_t5_model_trained.pth"
TEMP_SAVE_PATH ="/content/drive/MyDrive/nlp-project /temporal-folder/model-s.pth" 

def train(epoch, best_val_loss):
    model.train()
    total_loss = 0.
    for batch_index, batch in enumerate(train_loader):
        data, target = batch
        optimizer.zero_grad()
        masked_labels = mask_label_padding(target['input_ids'])
        output = model(
            input_ids=data['input_ids'],
            attention_mask=data['attention_mask'],
            labels=masked_labels
        )
        loss = output[0]
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        total_loss += loss.item()
        if batch_index % LOG_INTERVAL == 0 and batch_index > 0:
            cur_loss = total_loss / LOG_INTERVAL
            print('| epoch {:3d} | ' 
                  '{:5d}/{:5d} batches | '
                  'loss {:5.2f}'.format(
                    epoch, 
                    batch_index, len(train_loader), 
                    cur_loss))
            save(
                TEMP_SAVE_PATH,
                epoch, 
                model.state_dict(), 
                optimizer.state_dict(), 
                best_val_loss
            )
            total_loss = 0

def evaluate(eval_model, data_loader):
    eval_model.eval()
    total_loss = 0.
    with torch.no_grad():
        for batch_index, batch in enumerate(data_loader):
            data, target = batch
            masked_labels = mask_label_padding(target['input_ids'])
            output = eval_model(
                input_ids=data['input_ids'],
                attention_mask=data['attention_mask'],
                labels=masked_labels
            )
            total_loss += output[0].item()
    return total_loss / len(data_loader)

def mask_label_padding(labels):
    MASK_ID = -100
    labels[labels==tokenizer.pad_token_id] = MASK_ID
    return labels

def save(path, epoch, model_state_dict, optimizer_state_dict, loss):
    torch.save({
            'epoch': epoch,
            'model_state_dict': model_state_dict,
            'optimizer_state_dict': optimizer_state_dict,
            'best_loss': loss,
            }, path)

def load(path):
    return torch.load(path)

def print_line():
    LINE_WIDTH = 60
    print('-' * LINE_WIDTH)

In [ ]:
#TO START TRAINING  
best_val_loss = float("inf")
best_model = None

val_loss = evaluate(model, valid_loader)
print_line()
print('| Before training | valid loss {:5.2f}'.format(
    val_loss)
)
print_line()

for epoch in range(2, EPOCHS + 1):

    train(epoch,best_val_loss)
    val_loss = evaluate(model, valid_loader)
    print_line()
    print('| end of epoch {:3d} | valid loss {:5.2f}'.format(
        epoch,
        val_loss)
    )
    print_line()

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        best_model = model
        save(
             SAVED_MODEL_PATH,
             epoch, 
             model.state_dict(), 
             optimizer.state_dict(), 
             best_val_loss
        )
        print("| Model saved.")
        print_line()

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
def mask_label_padding(labels):
    MASK_ID = -100
    labels[labels==tokenizer.pad_token_id] = MASK_ID
    return labels

In [69]:
#FOR CALCULATING loss OF MODEL IN VALID DATASET
def evaluate(eval_model, data_loader):
    eval_model.eval()
    total_loss = 0.
    with torch.no_grad():
        for batch_index, batch in enumerate(data_loader):
            data, target = batch
            masked_labels = mask_label_padding(target['input_ids'])
            output = eval_model(
                input_ids=data['input_ids'],
                attention_mask=data['attention_mask'],
                labels=masked_labels
            )
            total_loss += output[0].item()
    return total_loss / len(data_loader)

In [70]:
def print_line():
    LINE_WIDTH = 60
    print('-' * LINE_WIDTH)
accuracy = evaluate(model, valid_loader)
print_line()
print('| After 7 epoch training | loss on valid set: {:5.2f}%'.format(accuracy))
print_line()

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2179: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


------------------------------------------------------------
| After 7 epoch training | loss on valid set:  2.54%
------------------------------------------------------------


In [67]:
text='At the moment, the most popular programming languages used in contests areC++, Python and Java. For example, in Google Code Jam 2017, among the best3,000 participants, 79 % used C++, 16 % used Python and 8 % used Java [29].Some participants also used several languages.Many people think that C++ is the best choice for a competitive programmer,and C++ is nearly always available in contest systems. '

In [65]:
%cd /content/question_generator

/content/question_generator


In [68]:
#for genrating question and also answers for given text
#cd /content/question_generator
from questiongenerator import QuestionGenerator
qg = QuestionGenerator()
qg.generate(text, num_questions=10)


Some weights of the model checkpoint at iarfmoose/t5-base-question-generator were not used when initializing T5ForConditionalGeneration: ['decoder.block.0.layer.1.EncDecAttention.relative_attention_bias.weight']
- This IS expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing T5ForConditionalGeneration from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Generating questions...



/usr/local/lib/python3.7/dist-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.5) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.1). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


Evaluating QA pairs...



[{'answer': 'Many people think that C++ is the best choice for a competitive programmer,and C++ is nearly always available in contest systems.',
  'question': 'What is the best programming language for a competitive programmer?'},
 {'answer': [{'answer': '8 %', 'correct': False},
   {'answer': 'Google', 'correct': False},
   {'answer': 'Java', 'correct': False},
   {'answer': 'Python', 'correct': True}],
  'question': 'What language was used in Google Code Jam 2017?'},
 {'answer': [{'answer': 'C++', 'correct': False},
   {'answer': 'Java', 'correct': True},
   {'answer': 'Google', 'correct': False},
   {'answer': 'Python', 'correct': False}],
  'question': 'What languages are the most popular in programming contests?'},
 {'answer': 'At the moment, the most popular programming languages used in contests areC++, Python and Java.',
  'question': 'What languages are used in Google Code Jam?'},
 {'answer': [{'answer': 'best3,000', 'correct': False},
   {'answer': 'C++', 'correct': True},
  